In [28]:
import tensorflow as tf
import matplotlib.pyplot as plt 

from tensorflow.keras.optimizers import Adam

from gpu.negfc_models import get_model_new, get_model_angle
from gpu.losses import reduce_std, wrapper, reduce_std_angle
from gpu.data import load_data
import numpy as np

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
%%time
root='./data/fake/'
dataset, cube_shp, init_xy, init_flux = load_data(root, 
                                                  bkg_sigma=2, 
                                                  ncomp=2, 
                                                  num_peaks=3,
                                                  normalize=False,
                                                  psf_pos=0,
                                                  )

CPU times: user 12.6 s, sys: 506 ms, total: 13.1 s
Wall time: 6.69 s


In [30]:
init_x = init_xy[:, 0] - cube_shp[-1]/2
init_y = init_xy[:, 1] - cube_shp[-2]/2


init_radius = tf.sqrt(tf.pow(init_y, 2)+tf.pow(init_x, 2))

angle = tf.atan2(init_y, init_x) # radians
angle = angle/tf.constant(np.pi)*180.  # degrees
init_theta = tf.math.mod(angle, 360.) # bound up to 1 circumference  

In [31]:
init_theta.numpy(), init_radius.numpy(), init_flux

(array([120.19304, 135.65742], dtype=float32),
 array([249.15666,  74.78336], dtype=float32),
 array([131.41001151,   1.91131941]))

In [32]:
model = get_model_angle(init_radius, init_theta, init_flux, cube_shp)

In [33]:
angle = 120. 
r = 250.

In [34]:
loss_fn = wrapper(reduce_std_angle, nfwhm=2, debug=False, minimize='std')
model.compile(loss_fn=loss_fn, 
              optimizer=tf.keras.optimizers.Adam(5))

In [35]:
# from gpu.losses import reduce_std_angle

# %load_ext autoreload
# %autoreload 2

# for x, y_true in dataset:
#     y_pred = model(x)
#     val = loss_fn(y_true, y_pred)
#     fig, axes = plt.subplots(1, 2, dpi=300, sharex=True, sharey=True)

#     axes[0].imshow(y_true['cube'][0, 0])
#     axes[1].imshow(tf.reduce_sum(val, 1)[0])

#     fake   = y_pred[0]
#     radius = y_pred[1]
#     theta  = y_pred[2] 

#     n_candidates = tf.shape(fake)[0]
#     n_frames = tf.shape(fake)[1]
#     w = tf.shape(fake)[2]
#     h = tf.shape(fake)[3]
    
#     cube = tf.tile(y_true['cube'], [n_candidates, 1, 1, 1])
#     cube = tf.expand_dims(cube, -1)

#     injected = fake + cube
    
#     # Create a grid of x and y coordinates
#     x_coords = tf.range(w, dtype=tf.float32)
#     y_coords = tf.range(h, dtype=tf.float32)
#     X, Y = tf.meshgrid(x_coords, y_coords)
#     Xf = tf.tile(tf.expand_dims(X, 0), [n_frames, 1, 1])
#     Yf = tf.tile(tf.expand_dims(Y, 0), [n_frames, 1, 1])
    
#     def fn(currr, currtheta):
#         rot_theta = currtheta - y_true['rot_angles']# rotate angles 
#         rot_theta = tf.experimental.numpy.deg2rad(rot_theta)
#         x0 = tf.multiply(currr, tf.cos(rot_theta)) + tf.cast(h/2, tf.float32)
#         y0 = tf.multiply(currr, tf.sin(rot_theta)) + tf.cast(w/2, tf.float32)   
#         x0 = tf.reshape(x0, [-1, 1, 1])
#         y0 = tf.reshape(y0, [-1, 1, 1])

#         distances = tf.sqrt(tf.pow(Xf - x0, 2) + tf.pow(Yf - y0, 2))
#         mask = tf.where(distances <= 2*y_true['fwhm'], 1., 0.)
#         return mask
    
#     mask_all = tf.map_fn(lambda x: fn(x[0], x[1]),
#                      (radius, theta),
#                      fn_output_signature=(tf.float32))      
#     sum_fake = tf.reduce_sum(val, 1)
#     plt.imshow(sum_fake[0])

In [ ]:
## %%time
es = tf.keras.callbacks.EarlyStopping(
    monitor='loss', patience=20,# min_delta=1e-5
)
hist = model.fit(dataset, epochs=100000, callbacks=[es])

Epoch 1/100000
std
1/1 [==============================] - 4s 4s/step - loss: 117191.9453
Epoch 2/100000
1/1 [==============================] - 2s 2s/step - loss: 115807.8906
Epoch 3/100000
1/1 [==============================] - 2s 2s/step - loss: 114454.2734
Epoch 4/100000
1/1 [==============================] - 2s 2s/step - loss: 113131.6641
Epoch 5/100000
1/1 [==============================] - 2s 2s/step - loss: 111838.8359
Epoch 6/100000
1/1 [==============================] - 2s 2s/step - loss: 110571.9375
Epoch 7/100000
1/1 [==============================] - 2s 2s/step - loss: 109324.7891
Epoch 8/100000
1/1 [==============================] - 2s 2s/step - loss: 108091.2500
Epoch 9/100000
1/1 [==============================] - 2s 2s/step - loss: 106868.1016
Epoch 10/100000
1/1 [==============================] - 2s 2s/step - loss: 105655.8672
Epoch 11/100000
1/1 [==============================] - 2s 2s/step - loss: 104457.5703
Epoch 12/100000
1/1 [==============================] - 2s 2

In [ ]:
plt.plot(hist.history['loss'])
plt.yscale('log')

In [ ]:
import gpu.fake_comp as tfnegfc
import tensorflow_addons as tfa
from vip_hci.fits import open_fits
import numpy as np

In [ ]:
cube_route = os.path.join(root, 'center_im.fits')
cube       = open_fits(cube_route, header=False) 
if tf.rank(cube) < 4:
    cube = cube[None,...]
psf_route = os.path.join(root, 'median_unsat.fits')
psf       = open_fits(psf_route, header=False)
if psf.shape[-1] % 2 == 0:
    psf_reduced = psf[...,:-1,:-1]  
    psf = np.zeros_like(psf_reduced)
    for i, p_pos in enumerate(psf_reduced):
        for k, p_lambda in enumerate(p_pos):
            psf[i][k] = tfa.image.translate(p_lambda, [0.5, 0.5])
    
ra_route = os.path.join(root, 'rotnth.fits')
rot_ang  = open_fits(ra_route, header=False)
rot_ang  = -rot_ang

lambda_ch = 0 # channel to use
psf_pos   = 0 # psf position

results = tfnegfc.adjust_gaussian(psf[lambda_ch, psf_pos])
fwhm_sphere  = tf.reduce_mean(results['fwhm'])
centered_psf = tfnegfc.center_cube(psf[lambda_ch], fwhm_sphere)
normalized_psf = tfnegfc.normalize_psf(centered_psf, fwhm=fwhm_sphere)
patch = tfnegfc.create_patch(cube[0, 0], normalized_psf[0])

In [ ]:
r_fguess = model.trainable_variables[0]
t_fguess = model.trainable_variables[1]
f_fguess = model.trainable_variables[2]

rot_theta = t_fguess # rotate angles 
rot_theta = tf.experimental.numpy.deg2rad(rot_theta)

x_fguess = tf.multiply(r_fguess, tf.cos(rot_theta)) + tf.cast(cube.shape[-1]/2, tf.float32)
y_fguess = tf.multiply(r_fguess, tf.sin(rot_theta)) + tf.cast(cube.shape[-2]/2, tf.float32) 

In [ ]:
from vip_hci.fm.negfc_fmerit import cube_inject_companions

In [ ]:
r_fguess.numpy(), t_fguess.numpy(), f_fguess.numpy()

In [ ]:
%%time
adi_image = tfnegfc.apply_adi(cube[0], 
                              rot_ang, 
                              out_size=cube[0].shape, 
                              ncomp=2, 
                              derotate='tf', 
                              return_cube=False)

injected_cube = tfnegfc.inject_fake(x_fguess[0], 
                                    y_fguess[0], 
                                    f_fguess[0], 
                                    np.zeros_like(cube[0]), 
                                    patch, 
                                    rot_ang)

adi_image_fake = tfnegfc.apply_adi(injected_cube, 
                                   rot_ang, 
                                   out_size=injected_cube.shape, 
                                   ncomp=2, 
                                   derotate='tf', 
                                   return_cube=False)

In [ ]:
fig, axes = plt.subplots(3, 1, dpi=300, figsize=(6,6), sharex=True, sharey=True)
axes[0].imshow(adi_image)
axes[0].scatter(x_fguess[0], y_fguess[0], marker='x', s=1)
axes[0].scatter(init_xy[:, 0][0], init_xy[:, 1][0], marker='x', s=1)

axes[1].imshow(adi_image_fake)
axes[2].imshow(adi_image+adi_image_fake)
axes[2].set_xlim(300,500)
axes[2].set_ylim(700,800)